In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from globalfunctions import *
%matplotlib inline

tickers = ['SPY','EFA','IEF','GLD']

In [2]:
# format epoch time to datetime
def convertTime(dtEpoch):
    formattedDate = datetime.datetime.fromtimestamp(dtEpoch).strftime('%Y-%m-%d')
    return formattedDate

# Get Historical Data
def GetModalData(tickers = []):
    lst_data = {}
    key =  getData("SELECT [keyValue] FROM [fattailinvestorweb].[dbo].[tblAPIKeys] where pkAPIID = 1")
    for t in tickers:
        # Get Price data
        priceHistory = get_price_history(key['keyValue'][0],symbol=t, period=1, periodType='year', frequencyType='weekly')
        # focus on dict item with price history
        data = priceHistory['candles']
        # loop through dictionary to get close price data
        lst_price = []
        lst_date = []
        for d in data:
            close = (d['close']+d['high'] + d['low'])/3
            date = convertTime((d['datetime']/1000)+(86400*5))
            lst_price.append(close)
            lst_date.append(date)
        if 'Date' not in lst_data:
            lst_data['Date'] = lst_date
        lst_data[t] = lst_price
        

    return lst_data

    

In [3]:
lst = GetModalData(tickers)


In [4]:
print(lst)

{'Date': ['2019-09-06', '2019-09-13', '2019-09-20', '2019-09-27', '2019-10-04', '2019-10-11', '2019-10-18', '2019-10-25', '2019-11-01', '2019-11-08', '2019-11-15', '2019-11-22', '2019-11-29', '2019-12-06', '2019-12-13', '2019-12-20', '2019-12-27', '2020-01-03', '2020-01-10', '2020-01-17', '2020-01-24', '2020-01-31', '2020-02-07', '2020-02-14', '2020-02-21', '2020-02-28', '2020-03-06', '2020-03-13', '2020-03-20', '2020-03-27', '2020-04-03', '2020-04-10', '2020-04-17', '2020-04-24', '2020-05-01', '2020-05-08', '2020-05-15', '2020-05-22', '2020-05-29', '2020-06-05', '2020-06-12', '2020-06-19', '2020-06-26', '2020-07-03', '2020-07-10', '2020-07-17', '2020-07-24', '2020-07-31', '2020-08-07', '2020-08-14', '2020-08-21', '2020-08-28'], 'SPY': [295.3833333333333, 299.84, 299.44, 296.31, 292.5416666666667, 294.50333333333333, 297.9266666666667, 300.765, 304.68666666666667, 308.21666666666664, 310.3, 310.90333333333336, 313.92333333333335, 312.43666666666667, 316.26666666666665, 319.986133333333

In [5]:
df = pd.DataFrame(lst,columns=['Date','SPY','EFA','IEF','GLD'])
df.tail()

,Date,SPY,EFA,IEF,GLD
47,2020-07-31,324.263333,62.723333,122.603333,184.330000
48,2020-08-07,332.393333,63.476667,122.693333,189.819967
49,2020-08-14,335.710000,64.676667,121.800000,184.730000
50,2020-08-21,338.140000,64.673333,121.956133,183.966667
51,2020-08-28,346.916800,65.343333,121.590000,183.053333


In [6]:
dfDic = {}
for t in tickers:
    colIndex = df.columns.get_loc(t)
    dfDic[t] = colIndex
print(dfDic)

{'SPY': 1, 'EFA': 2, 'IEF': 3, 'GLD': 4}


In [7]:
def CalculateROC(dataFrame, dfDic,N):
    df = dataFrame.copy()
    for key in dfDic:
        df[key+"_ROC"] = np.NaN
        rocIndex = df.columns.get_loc(key+"_ROC")
        for i,j in df.iterrows():
            if i>=N-1:
                df.iloc[i,rocIndex] = (df.iloc[i,dfDic[key]]/df.iloc[i-N,dfDic[key]])-1
    return df
                    
        

In [8]:
df = CalculateROC(df,dfDic,43)
df.tail()

,Date,SPY,EFA,IEF,GLD,SPY_ROC,EFA_ROC,IEF_ROC,GLD_ROC
47,2020-07-31,324.263333,62.723333,122.603333,184.330000,0.108435,-0.021909,0.083379,0.307460
48,2020-08-07,332.393333,63.476667,122.693333,189.819967,0.128657,-0.022483,0.090656,0.350968
49,2020-08-14,335.710000,64.676667,121.800000,184.730000,0.126821,-0.020026,0.087903,0.317023
50,2020-08-21,338.140000,64.673333,121.956133,183.966667,0.124266,-0.032609,0.091916,0.300761
51,2020-08-28,346.916800,65.343333,121.590000,183.053333,0.138602,-0.034145,0.086401,0.292111


In [9]:
dfSummary = {}
for t in tickers:
    colTitle = t+'_ROC'
    roc = df[colTitle][51]
    dfSummary[colTitle] = roc
print(dfSummary)

{'SPY_ROC': 0.13860184232982498, 'EFA_ROC': -0.0341446590461173, 'IEF_ROC': 0.08640100071479617, 'GLD_ROC': 0.2921107738641442}
